# Path

In [18]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Imports

In [19]:
from helpers import column_formatter, utf8_convert
from names import get_first_name, get_last_name
from config import data_path as path
from random import choice
from json import dump
import pandas as pd
import numpy as np

import openpyxl

# Read

In [27]:
appeals = pd.read_excel(path + '/1-raw/basvuru.xlsx')

sheetnames = openpyxl.load_workbook(path+'1-raw/yoklama.xlsx').sheetnames

sheets = dict()
for k in sheetnames:
    sheets[k] = pd.read_excel(path+'1-raw/yoklama.xlsx', sheet_name=k)
    print(list(sheets[k].columns))

['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'EĞİTİMCİSİ', 'CİNSİYET']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM', 'SOYİSİM ', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM ', 'SOYİSİM', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİM', 'SOYİSİM ', 'CİNSİYET', 'EĞİTİMCİ']
['EĞİTİM ', 'TARİH', 'GÜN', 'SAAT', 'İSİ

# Appeals

## Fake Names

In [21]:
fake_names = dict()

appeals['İsim - Soy isim'] = column_formatter(appeals['İsim - Soy isim']) 

for name in appeals['İsim - Soy isim']:
    fake_names[name] = get_first_name() + " " + get_last_name()

fake_list = list()

for name in appeals['İsim - Soy isim']:
    fake_list.append(fake_names[name])

appeals['İsim - Soy isim'] = fake_list

## Fake Numbers

In [22]:
nums = list(map(lambda x : str(x), [0,1,2,3,4,5,6,7,8,9]))


appeals['Cep Telefonunuz'] = list(map(lambda x : "".join([letter.replace(letter, choice(nums)) if letter.isnumeric() else letter for letter in x]) , [list(str(num)) for num in appeals['Cep Telefonunuz']]))

## Fake Mails

In [23]:
mails = ["kapsulteknoloji.org", "gmail.com", "outlook.com", "qraxiss.com"]
appeals['E-mail Adresiniz'] = ["@".join([nick, choice(mails)]) if isinstance(actual_mail, str) else float("NaN") for nick, actual_mail in zip(column_formatter(appeals['İsim - Soy isim']), appeals['E-mail Adresiniz'])]

# Rolls

In [24]:
sheets['finans'].drop("MAİL", axis=1, inplace=True)

In [25]:
for df in sheets.values():
    names = column_formatter(df[df.columns[4]] + df[df.columns[5]])

    fake_names2 = list()

    if 'EĞİTİMCİSİ' in df.columns:
        df['EĞİTİMCİSİ'] = get_first_name() + " " + get_last_name() 
    else:
        df['EĞİTİMCİ'] = get_first_name() + " " + get_last_name()

    for name in names:
        if name in fake_names:
            fake_names2.append(fake_names[name])

        else:
            fake_names2.append(get_first_name() + " " + get_last_name())

    df[[df.columns[4], df.columns[5]]] = pd.DataFrame(list(pd.DataFrame(fake_names2)[0].str.split(' ')))

# Write

In [26]:
with pd.ExcelWriter(path+'1-raw/yoklama.xlsx', mode='w') as writer:
    for sheet, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

with pd.ExcelWriter(path+'1-raw/basvuru.xlsx', mode='w') as writer:
    appeals.to_excel(writer, index=False)